# Course 2 Assignment 4

## Potential Moderator

For this assignment we are to perform either an ANOVA, Chi-square or Pearson correlation and see if the result is affected by the presence of a moderator variable.

i.e. does one variable change the relationship we are studying

Since my data is all quantitative, I will use the Pearson correlation again.

In [1]:
# import needed libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline

## Loading and preparing the data

In [ ]:
# show all columns
pd.set_option('display.max_columns', None)
# loading the data from the local file
df = pd.read_csv('data/covid_data.csv')

In [ ]:
# prepare data
df.date = pd.to_datetime(df.date)
dfx = df.dropna(subset=['continent'])  # gets rid of summaries for 'world' and 'africa' etc, as I only want data for countries
# the columns I need for this task
cols = ['location', 'date', 'new_cases_per_million','new_deaths_per_million', 'people_fully_vaccinated', 'aged_70_older', 'human_development_index', 'population']
dfx = dfx[cols].dropna()  # getting rid of rows with empty data
# getting rid of rows where new cases and  deaths are below zero (due to error correction I guess?)
dfx = dfx[dfx.new_deaths_per_million >= 0]
dfx = dfx[dfx.new_cases_per_million >= 0]
# limiting it to 2021 which is when vaccinations really got started
dfx = dfx[dfx['date'].dt.year == 2021]
# so as to compare like with like, I'm keeping only countries with human development indices over 0.9
dfx = dfx[dfx.human_development_index > 0.9]
# calculating percentage of population fully vaccinated
dfx['percentage_fully_vaccinated'] = (dfx.people_fully_vaccinated/dfx.population) * 100
dfx.tail()

## Pearson Tests

Since age is known to be a major risk factor in covid-19 deaths, I will see if the percentage of the population that is over 70 is a moderating variable.

In [ ]:
dfx['aged_ranked'] = pd.cut(dfx.aged_70_older, 2, labels=['younger', 'older'])
dfx[dfx.aged_ranked == 'older'].aged_70_older.describe()

In [ ]:
dfx[dfx.aged_ranked == 'younger'].aged_70_older.describe()

So the two groups I will compare are the 'older' countries, who have between 12.9 and 18.5% of their population aged over 70, and the 'younger' countries, who have between 7.0 and 12.7% of their population aged over 70.

### original pearson tests

In [ ]:
print('association between new cases per million and percentage fully vaccinated')
r, p = stats.pearsonr(dfx.new_cases_per_million, dfx.percentage_fully_vaccinated)
print(f'Pearson’s correlation coefficient (r): {r}')
print(f'p-value: {p}')
if p > 0.05:
    print('This result is not significant.')
else:
    print('This result is significant')

In [ ]:
print('association between new deaths per million and percentage fully vaccinated')
r, p = stats.pearsonr(dfx.new_deaths_per_million, dfx.percentage_fully_vaccinated)
print(f'Pearson’s correlation coefficient (r): {r}')
print(f'p-value: {p}')
if p > 0.05:
    print('This result is not significant.')
else:
    print('This result is significant')

## new pearson tests

In [ ]:
df_old = dfx[dfx.aged_ranked == 'older']
df_young = dfx[dfx.aged_ranked == 'younger']
names = ['older', 'younger']
dfs = [df_old, df_young]

for df, name in zip(dfs, names):
    print(f'\nPearson correlation for {name} countries')
    print('\nassociation between new cases per million and percentage fully vaccinated')
    r, p = stats.pearsonr(df.new_cases_per_million, df.percentage_fully_vaccinated)
    print(f'Pearson’s correlation coefficient (r): {r}')
    print(f'p-value: {p}')
    if p > 0.05:
        print('This result is not significant.')
    else:
        print('This result is significant')
        
    print('\nassociation between new deaths per million and percentage fully vaccinated')
    r, p = stats.pearsonr(df.new_deaths_per_million, df.percentage_fully_vaccinated)
    print(f'Pearson’s correlation coefficient (r): {r}')
    print(f'p-value: {p}')
    if p > 0.05:
        print('This result is not significant.')
    else:
        print('This result is significant') 

## Plots

In [ ]:
sns.set(font_scale=1.3)
sns.set_style('ticks')
g = sns.lmplot(x="percentage_fully_vaccinated", y="new_cases_per_million", hue="aged_ranked", data=dfx,
               markers=["o", "x"], height=7, truncate=False, scatter_kws={'alpha':0.5})
_ = g.set(ylim=(0,2550), xlim=(0,60), ylabel='new cases per million people', xlabel='percentage of population vaccinated', 
      title='new cases per million people vs. percentage of population vaccinated')

In [ ]:
sns.set(font_scale=1.3)
sns.set_style('ticks')
g = sns.lmplot(x="percentage_fully_vaccinated", y="new_deaths_per_million", hue="aged_ranked", data=dfx,
               markers=["o", "x"], height=7, truncate=False, scatter_kws={'alpha':0.5})
_ = g.set(ylim=(0,30), xlim=(0,60), ylabel='new deaths per million people', xlabel='percentage of population vaccinated', 
      title='new deaths per million people vs. percentage of population vaccinated')

## Summary

While the original Pearson test showed no corelation between new cases per million people and the percentage of population vaccinated, this is not true in the 'older countries' subset of my sample (Countries with a Human Development Index greater than 0.9, in 2021).

For those countries with more than 12.9% of their population aged 70 or older, there is a weak positive correlation between the number of new cases per million people and the percentage of population vaccinated (r = 0.162, p < 0.001). This suggests that the more people vaccinated, the more people infected with covid-19! This seems highly unlikely, and I assume there is something else going on. If you look at the graphs, you can see that the data for the older countries is concentrated in the 0 - 10 % vaccinated region, suggesting not much vaccination has actually occured in these countries, and so perhaps we don't currently have enough data to be drawing proper conclusions, I would strongly suggest in this case that correlation does not equal causation!

With that in mind, although the Pearson correlation test showed a stronger correlation in the older countries between the number of new deaths per million people and the percentage of population vaccinated (r = -0.196, p < 0.001) than in the younger countries (r = -0.106, p < 0.001), or overall (r = -0.138, p < 0.001), the issues stated above suggest we should interpret that with caution.